# Hướng Dẫn Tư Duy: Phân Tích XML Bằng Cây Tổng Quát

## 1. Bản Chất Bài Toán
Chúng ta cần chuyển đổi dữ liệu từ dạng **Chuỗi tuyến tính** (Linear String - file XML) sang dạng **Cấu trúc phân cấp** (Hierarchical Structure - Cây tổng quát).

* **Input:** Một chuỗi văn bản dài vô tận (`<store><book>...</book></store>`).
* **Output:** Một đối tượng Root Node chứa các Node con, các Node con lại chứa các Node cháu...


## 2. Mô Hình Hóa (Mapping)
Để code được, ta cần quy đổi các thành phần của XML sang thành phần của Cây:

| Thành phần XML | Thành phần Cây (General Tree) |
| :--- | :--- |
| **Thẻ mở** (`<tag>`) | Hành động **tạo mới một Nút (Node)**. |
| **Thẻ lồng nhau** | Quan hệ **Cha - Con** (Parent - Child). |
| **Thẻ ngang hàng** | Quan hệ **Anh em** (Siblings). |
| **Văn bản** (`text`) | Dữ liệu (`data`) lưu bên trong Nút. |
| **Thẻ đóng** (`</tag>`) | Hành động **kết thúc nhánh hiện tại**, quay về Cha. |


## 3. Thiết Kế Cấu Trúc Dữ Liệu (Data Structure)
Do số lượng thẻ con là không xác định, ta sử dụng mô hình **Danh sách liên kết các con**.

```python
class Node:
    String name      # Tên thẻ (VD: "book")
    String content   # Nội dung (VD: "Harry Potter")
    List<Node> children  # Danh sách chứa các nút con

## 4. Chiến Lược Thuật Toán (The Logic)

Làm sao để máy tính đọc một chuỗi văn bản tuần tự (từ trái qua phải) mà lại xây dựng được một cây phân cấp? Chìa khóa nằm ở việc sử dụng **Ngăn xếp (Stack)** để quản lý "ngữ cảnh" (Context).

### Cơ chế hoạt động

Stack sẽ luôn lưu giữ **đường dẫn từ Gốc đến nút hiện tại** đang được xử lý. Đỉnh của Stack (Top) luôn trả lời cho câu hỏi: *"Nút nào là cha của dữ liệu tôi đang đọc?"*

### Quy tắc xử lý 3 bước

Thuật toán sẽ duyệt qua từng thành phần (token) của file XML và thực hiện hành động dựa trên loại thẻ:

#### 1. Gặp Thẻ Mở (Open Tag) - Ví dụ: `<book>`
Đây là tín hiệu bắt đầu một đối tượng mới.
* **Hành động 1:** Khởi tạo một Node mới (ví dụ: Node `book`).
* **Hành động 2:** Kiểm tra Stack:
    * Nếu Stack có phần tử: Nút ở đỉnh Stack chính là **Cha**. Ta thêm Node mới vào danh sách `children` của Cha.
    * Nếu Stack rỗng: Node mới chính là **Gốc (Root)** của cây.
* **Hành động 3:** **Đẩy (Push)** Node mới vào Stack. (Để nó trở thành Cha của các thẻ con hoặc nội dung sắp tới).

#### 2. Gặp Nội Dung (Text Content) - Ví dụ: `"Harry Potter"`
Đây là dữ liệu thuộc về nút đang mở.
* **Hành động:** Xem (Peek) nút đang ở đỉnh Stack và gán chuỗi văn bản vào thuộc tính dữ liệu của nút đó.

#### 3. Gặp Thẻ Đóng (Close Tag) - Ví dụ: `</book>`
Đây là tín hiệu kết thúc một đối tượng.
* **Hành động:** **Lấy ra (Pop)** nút ở đỉnh Stack.
* **Ý nghĩa:** Chúng ta đã xử lý xong nút con này, bây giờ "lùi bước" về nút Cha để tiếp tục xử lý các anh em khác (nếu có).

---

### Ví dụ mô phỏng (Trace)

Giả sử input là: `<store><book>Code</book></store>`

| Bước | Đọc được | Hành động Logic | Trạng thái Stack (Đỉnh nằm bên phải) | Cấu trúc Cây hình thành |
| :--- | :--- | :--- | :--- | :--- |
| 1 | `<store>` | Tạo node `store`. Stack rỗng $\rightarrow$ Là Root. Push `store`. | `[store]` | `store` |
| 2 | `<book>` | Tạo node `book`. Cha là `store` (đỉnh stack). Push `book`. | `[store, book]` | `store` $\rightarrow$ `book` |
| 3 | `"Code"` | Gán text "Code" cho đỉnh stack (`book`). | `[store, book]` | `store` $\rightarrow$ `book("Code")` |
| 4 | `</book>` | Gặp thẻ đóng. Pop stack (lấy `book` ra). | `[store]` | (Đã xong nhánh `book`, quay về `store`) |
| 5 | `</store>` | Gặp thẻ đóng. Pop stack (lấy `store` ra). | `[]` | (Hoàn thành cây) |

## 5. Giả Mã (Pseudo-code)
Khởi tạo: root = Null, stack = Empty

Duyệt từng token trong chuỗi XML:
    
    Trường hợp là Thẻ Mở (<name>):
        node = New Node(name)
        Nếu root == Null: root = node
        Nếu stack có phần tử:
            parent = stack.peek()  // Xem đỉnh stack
            parent.children.add(node)
        stack.push(node)           // Đưa node mới lên đỉnh làm cha hiện tại

    Trường hợp là Nội Dung (text):
        current_node = stack.peek()
        current_node.content = text

    Trường hợp là Thẻ Đóng (</name>):
        stack.pop()                // Xong việc, rút lui về cha cũ

In [2]:
import sys

# ==========================================
# PHẦN 1: ĐỊNH NGHĨA CẤU TRÚC CÂY TỔNG QUÁT
# ==========================================
class GeneralTreeNode:
    """
    Đại diện cho một nút trong cây tổng quát.
    Mỗi nút đại diện cho một thẻ (tag) trong XML.
    """
    def __init__(self, tag_name):
        self.tag_name = tag_name        # Tên thẻ (VD: store, book)
        self.text_content = ""          # Nội dung văn bản bên trong thẻ
        self.children = []              # Danh sách chứa các nút con (List of Children)

    def add_child(self, child_node):
        """Thêm một nút con vào danh sách quản lý"""
        self.children.append(child_node)

    def __repr__(self):
        """Hiển thị debug đơn giản"""
        return f"<Node: {self.tag_name}>"

    def display_tree(self, level=0):
        """
        Hàm đệ quy để in toàn bộ cây ra màn hình (Visualization).
        Sử dụng kỹ thuật thụt đầu dòng để thể hiện cấp độ cha-con.
        """
        # Tạo khoảng trắng thụt đầu dòng (2 dấu cách cho mỗi cấp)
        indent = "  " * level
        
        # Chỉ hiển thị nội dung nếu nó không rỗng
        content_display = f": {self.text_content.strip()}" if self.text_content.strip() else ""
        
        # In ra: Khoảng trắng + <Tên thẻ> + Nội dung
        print(f"{indent}- <{self.tag_name}>{content_display}")

        # ĐỆ QUY: Gọi tiếp hàm này cho tất cả các con của nó
        for child in self.children:
            child.display_tree(level + 1)


# ==========================================
# PHẦN 2: THUẬT TOÁN PARSER (DÙNG STACK)
# ==========================================
def parse_xml_to_tree(xml_string):
    """
    Chuyển đổi chuỗi XML thành Cây tổng quát.
    Input: Chuỗi String XML
    Output: Nút Gốc (Root) của cây
    """
    
    # BƯỚC 1: TOKENIZATION (Tách chuỗi thành các phần nhỏ)
    # Thêm khoảng trắng quanh dấu < và > để dễ dàng dùng hàm split()
    # VD: "<book>A</book>" -> " <book> A  </book> "
    formatted_xml = xml_string.replace("<", " <").replace(">", "> ")
    tokens = formatted_xml.split()

    # Khởi tạo Stack và Root
    root = None
    stack = [] # Stack lưu trữ các Node đang xử lý (chưa gặp thẻ đóng)

    # BƯỚC 2: DUYỆT QUA TỪNG TOKEN
    for token in tokens:
        
        # --- TRƯỜNG HỢP 1: THẺ ĐÓNG (VD: </book>) ---
        if token.startswith("</"):
            # Logic: Gặp thẻ đóng nghĩa là nhánh này đã xong.
            # Ta rút lui (backtrack) bằng cách pop khỏi stack.
            if stack:
                stack.pop()

        # --- TRƯỜNG HỢP 2: THẺ MỞ (VD: <book>) ---
        elif token.startswith("<"):
            # Lấy tên thẻ (bỏ dấu < và >). VD: "<book>" -> "book"
            tag_name = token[1:-1]
            
            # Tạo một nút mới
            new_node = GeneralTreeNode(tag_name)

            # Nếu đây là nút đầu tiên, nó là Root
            if root is None:
                root = new_node
            
            # Nếu Stack không rỗng, nghĩa là nút mới này là CON của nút trên đỉnh Stack
            if stack:
                current_parent = stack[-1] # Peek: Xem ai đang là cha
                current_parent.add_child(new_node) # Nối cha - con

            # Đẩy nút mới vào Stack để nó làm cha của các phần tử tiếp theo
            stack.append(new_node)

        # --- TRƯỜNG HỢP 3: NỘI DUNG VĂN BẢN (VD: Harry Potter) ---
        else:
            # Logic: Nếu có nội dung, nó thuộc về nút đang mở (đỉnh Stack)
            if stack:
                current_node = stack[-1]
                # Cộng dồn chuỗi (đề phòng text có dấu cách bị tách ra)
                current_node.text_content += token + " "

    return root


# ==========================================
# PHẦN 3: DRIVER CODE (CHẠY THỬ)
# ==========================================
if __name__ == "__main__":
    # 1. Dữ liệu giả lập (Một đoạn XML phức tạp nhiều tầng)
    raw_xml_data = """
    <thuvien>
        <khoa_hoc>
            <sach>
                <ten>Cau Truc Du Lieu</ten>
                <tac_gia>Nguyen Van A</tac_gia>
            </sach>
            <sach>
                <ten>Giai Thuat</ten>
                <gia>100k</gia>
            </sach>
        </khoa_hoc>
        <tieu_thuyet>
            <truyen>
                <ten>De Men Phieu Luu Ky</ten>
            </truyen>
        </tieu_thuyet>
    </thuvien>
    """

    print("--- BẮT ĐẦU PHÂN TÍCH XML ---")
    
    # 2. Gọi hàm phân tích
    tree_root = parse_xml_to_tree(raw_xml_data)

    # 3. Hiển thị kết quả
    if tree_root:
        print(f"\n[Thành công] Đã xây dựng xong cây. Gốc là: {tree_root.tag_name}\n")
        print("--- CẤU TRÚC CÂY HIỂN THỊ ---")
        tree_root.display_tree()
    else:
        print("[Lỗi] Không thể dựng cây (XML rỗng hoặc lỗi).")

--- BẮT ĐẦU PHÂN TÍCH XML ---

[Thành công] Đã xây dựng xong cây. Gốc là: thuvien

--- CẤU TRÚC CÂY HIỂN THỊ ---
- <thuvien>
  - <khoa_hoc>
    - <sach>
      - <ten>: Cau Truc Du Lieu
      - <tac_gia>: Nguyen Van A
    - <sach>
      - <ten>: Giai Thuat
      - <gia>: 100k
  - <tieu_thuyet>
    - <truyen>
      - <ten>: De Men Phieu Luu Ky
